In [1]:
import requests_cache
import time
from requests.exceptions import HTTPError, ConnectionError, Timeout, RequestException
from requests.adapters import HTTPAdapter
from urllib3 import Retry
from datetime import timedelta
from pprint import PrettyPrinter
import pandas as pd



class APIManager:
    """
    APIManager class

    This singleton class interfaces with the US Census API.
    """

    def __init__(self):
        """

        Constructor

        Sets up the connection to the US Census API.

        Returns:
            An instance of the APIManager
        """
        session = requests_cache.CachedSession(
            "api-response-cache",
            backend="filesystem",
            expire_after=timedelta(days=1),
        )
        retry = Retry(total=5, backoff_factor=1)
        adapter = HTTPAdapter(max_retries=retry)
        base_url = "https://api.census.gov/data"
        session.mount(base_url, adapter)

        self.session = session
        self.base_url = base_url

    def api_call(self, url="", params=None):
        """
        api_call

        Handles the request to the US Census API.

        Arguments:
            - url (str): the url to request; appended onto the base_url
            - params (dict): the parameters to pass into the request

        Returns:
            The response data in json format
        """
        start = time.time()
        print(f"API call in progress...")

        try:
            response = self.session.get(self.base_url + url, params=params, timeout=10)
            response.raise_for_status()
        # TODO: handle errors separately
        except HTTPError as e:
            print(f"Failed API call (HTTPError): {e}")
            raise
        except ConnectionError as e:
            print(f"Failed API call (ConnectionError): {e}")
            raise
        except Timeout as e:
            print(f"Failed API call (Timeout): {e}")
            raise
        except RequestException as e:
            print(f"Failed API call (RequestException): {e}")
            raise
        else:
            elapsed = time.time() - start
            info = "from cache" if response.from_cache else f"took {round(elapsed)}s"
            print(f"Successful API call ({info}): {response.url}")

            return response.json()


api_manager = APIManager()


In [2]:
census_year = "2021"
api_key = 'e3061d8962ee2b9822717e18093c29337bca18df'
profile_URL = "/{}/acs/acs5/profile".format(census_year)

In [3]:
vars_json = profile_URL + "/variables.json"
variable_list = api_manager.api_call(vars_json)
var_list = [x for x,v in variable_list['variables'].items() if x not in ['for','in','ucgid']]

API call in progress...
Successful API call (from cache): https://api.census.gov/data/2021/acs/acs5/profile/variables.json
